<a href="https://colab.research.google.com/github/rippedtuco/Musconet/blob/main/autoEnc_Abnormal_Knee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from google.colab import drive
from google.colab import files

def load_patient_data(main_directory):

    # Get the list of patient folders
    patient_folders = [folder for folder in os.listdir(main_directory)]

    march_data = []  # Initialize empty data frame for march data
    leg_extension_data = []  # Initialize empty data frame for leg extension data
    knee_flexion_data = []  # Initialize empty data frame for knee flexion data

    for patient_folder in patient_folders:
        patient_path = os.path.join(main_directory, patient_folder)  # Path to the patient folder

        march_files = [file for file in os.listdir(patient_path) if file.__contains__("mar")]
        march_patient_data = pd.concat([pd.read_csv(os.path.join(patient_path, file)) for file in march_files]) # Load CSV data for the patient
        march_data.append(march_patient_data)  # Append patient data to march data frame

        leg_extension_files = [file for file in os.listdir(patient_path) if file.__contains__("pie")]
        leg_extension_patient_data = pd.concat([pd.read_csv(os.path.join(patient_path, file)) for file in leg_extension_files]) # Load CSV data for the patient
        leg_extension_data.append(leg_extension_patient_data)  # Append patient data to leg extension data frame

        knee_flexion_files = [file for file in os.listdir(patient_path) if file.__contains__("sen")]
        knee_flexion_patient_data = pd.concat([pd.read_csv(os.path.join(patient_path, file)) for file in knee_flexion_files]) # Load CSV data for the patient
        knee_flexion_data.append(knee_flexion_patient_data)  # Append patient data to knee flexion data frame

    march_data = pd.concat(march_data)  # Concatenate all march data frames
    leg_extension_data = pd.concat(leg_extension_data)  # Concatenate all leg extension data frames
    knee_flexion_data = pd.concat(knee_flexion_data)  # Concatenate all knee flexion data frames

    
    return march_data,leg_extension_data,knee_flexion_data

# Example usage
drive.mount('/content/drive')
main_directory = "/content/drive/MyDrive/EMG/CSV_A"
b_march_data,b_leg_extension_data,b_knee_flexion_data = load_patient_data(os.path.join(main_directory))

column_index = 4
if column_index < len(b_march_data.columns):
    b_march_data = b_march_data.drop(b_march_data.columns[column_index], axis=1)

if column_index < len(b_leg_extension_data.columns):
    b_leg_extension_data = b_leg_extension_data.drop(b_leg_extension_data.columns[column_index], axis=1)

if column_index < len(b_knee_flexion_data.columns):
    b_knee_flexion_data = b_knee_flexion_data.drop(b_knee_flexion_data.columns[column_index], axis=1)

b_march_data_train,b_march_data_test=train_test_split(b_march_data,test_size=0.2)
b_leg_extension_data_train,b_leg_extension_data_test=train_test_split(b_leg_extension_data,test_size=0.2)
b_knee_flexion_data_train,b_knee_flexion_data_test=train_test_split(b_knee_flexion_data,test_size=0.2)

num_features=4

input_shape = (num_features,)

input_data = Input(shape=input_shape)
encoded = Dense(64, activation='elu')(input_data)
encoded = Dense(32, activation='elu')(encoded)
encoded = Dense(16, activation='elu')(encoded)


decoded = Dense(32, activation='elu')(encoded)
decoded = Dense(64, activation='elu')(decoded)
decoded = Dense(num_features, activation='linear')(decoded)

# Create the autoencoder model
autoencoder = Model(input_data, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder using your good EMG data
autoencoder.fit(b_knee_flexion_data_train, b_knee_flexion_data_train, epochs=10, batch_size=32)

# Create a new model for feature extraction
encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dense_18').output)

encoder_model.save('encoder_model_bad_knee_flexion.h5')
files.download('encoder_model_bad_knee_flexion.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/10
7177/7177 [==============================] - 21s 3ms/step - loss: 1.4604
Epoch 2/10
7177/7177 [==============================] - 18s 3ms/step - loss: 0.0164
Epoch 3/10
7177/7177 [==============================] - 18s 3ms/step - loss: 0.0110
Epoch 4/10
7177/7177 [==============================] - 18s 3ms/step - loss: 0.0093
Epoch 5/10
7177/7177 [==============================] - 20s 3ms/step - loss: 0.0080
Epoch 6/10
7177/7177 [==============================] - 19s 3ms/step - loss: 0.0073
Epoch 7/10
7177/7177 [==============================] - 18s 3ms/step - loss: 0.0084
Epoch 8/10
7177/7177 [==============================] - 20s 3ms/step - loss: 0.0069
Epoch 9/10
7177/7177 [==============================] - 20s 3ms/step - loss: 0.0051
Epoch 10/10
7177/7177 [==============================] - 19s 3ms/step - loss: 0.0067


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>